In [1]:
from numpy import array, zeros, ones
from numpy.linalg import inv, solve
from math import pi as π, sin, cos

## Rigidez local

## Compatibilidad geometrica

## Grados de libertad

In [2]:
def K(k, a, b, gdl):
    T = zeros([6, 3*gdl])
    
    # elseif x == 1 || y == 1
    #     return "reacts not considered"
    # elseif x == 19 || y == 19
    #     return "reacts not considered"
    # elif x > 20 or y > 20
    #     return "index error"
    
    # x == 20 ? x-=1 : x
    # y == 20 ? y-=1 : y
    
    if a > gdl or b > gdl: return 'index error'
    
    if a != 0:
        T[0, 3*a - 3] = 1
        T[1, 3*a - 2] = 1
        #T[2, 3*a - 1] = 1
    if b != 0:
        T[3, 3*b - 3] = 1
        T[4, 3*b - 2] = 1
        #T[5, 3*b - 1] = 1
    
    print(T, '\n')
    return T.T@k@T

In [3]:
gdl = 4
a = 2
b = 4
t = K(ones([6, 6]), a, b, gdl)
print(t)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
